# Question 1

In [1]:
library(glmnet)

Loading required package: Matrix

Loaded glmnet 4.1-4



In [2]:
library(leaps)

In [3]:
library(pls)


Attaching package: ‘pls’


The following object is masked from ‘package:stats’:

    loadings




In [4]:
rm (list = ls())

In [5]:
prostate <- data.frame(read.table("http://statweb.stanford.edu/~tibs/ElemStatLearn/datasets/prostate.data"))

In [6]:
dim(prostate)
head(prostate, 10)

[1] 97 10

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa,train
,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<lgl>
1,-0.5798185,2.769459,50,-1.3862944,0,-1.386294,6,0,-0.4307829,TRUE
2,-0.9942523,3.319626,58,-1.3862944,0,-1.386294,6,0,-0.1625189,TRUE
3,-0.5108256,2.691243,74,-1.3862944,0,-1.386294,7,20,-0.1625189,TRUE
4,-1.2039728,3.282789,58,-1.3862944,0,-1.386294,6,0,-0.1625189,TRUE
5,0.7514161,3.432373,62,-1.3862944,0,-1.386294,6,0,0.3715636,TRUE
6,-1.0498221,3.228826,50,-1.3862944,0,-1.386294,6,0,0.7654678,TRUE
7,0.7371641,3.473518,64,0.6151856,0,-1.386294,6,0,0.7654678,FALSE
8,0.6931472,3.539509,58,1.5368672,0,-1.386294,6,0,0.8544153,TRUE
9,-0.7765288,3.539509,47,-1.3862944,0,-1.386294,6,0,1.0473190,FALSE


In [7]:
pro_1_train = sample(1:nrow(prostate), nrow(prostate)*0.80)

In [8]:
pro_1_train_data = prostate[pro_1_train,]
pro_1_test_data = prostate[-pro_1_train,]
dim(pro_1_train_data)
dim(pro_1_test_data)

[1] 77 10

[1] 20 10

In [9]:
best_subset = regsubsets(train ~., data = prostate, method = "exhaustive")

In [10]:
reg_summary = summary(best_subset)
reg_summary

Subset selection object
Call: regsubsets.formula(train ~ ., data = prostate, method = "exhaustive")
9 Variables  (and intercept)
        Forced in Forced out
lcavol      FALSE      FALSE
lweight     FALSE      FALSE
age         FALSE      FALSE
lbph        FALSE      FALSE
svi         FALSE      FALSE
lcp         FALSE      FALSE
gleason     FALSE      FALSE
pgg45       FALSE      FALSE
lpsa        FALSE      FALSE
1 subsets of each size up to 8
Selection Algorithm: exhaustive
         lcavol lweight age lbph svi lcp gleason pgg45 lpsa
1  ( 1 ) " "    " "     "*" " "  " " " " " "     " "   " " 
2  ( 1 ) " "    " "     " " " "  " " " " "*"     "*"   " " 
3  ( 1 ) " "    " "     "*" " "  " " " " "*"     "*"   " " 
4  ( 1 ) " "    " "     "*" " "  " " "*" "*"     "*"   " " 
5  ( 1 ) " "    " "     "*" "*"  " " "*" "*"     "*"   " " 
6  ( 1 ) " "    "*"     "*" "*"  " " "*" "*"     "*"   " " 
7  ( 1 ) " "    "*"     "*" "*"  "*" "*" "*"     "*"   " " 
8  ( 1 ) "*"    "*"     "*" "*"  "*" "

In [11]:
#computing AIC value is same as cp for regsubsets, so I computed cp value.
reg_summary$cp

[1] 0.4650383 1.3504137 0.4181620 1.3700022 2.3234078 4.1744942 6.1001435
[8] 8.0006358

Third is the best 

In [12]:
reg_summary$bic

[1]  6.040028  9.451005 10.895878 14.326838 17.745890 22.155045 26.646989
[8] 31.110819

First is the best

In [13]:
#5 fold
best_sub = regsubsets(train~.,data = pro_1_train_data,method="exhaustive")
train_errors = rep(NA,8)
test_errors = rep(NA,8)
Y_train = pro_1_train_data$train
Y_test = pro_1_test_data$train
train_pred_matrix = model.matrix(train~., data = pro_1_train_data)

In [14]:
test_pred_matrix = model.matrix(train~., data = pro_1_test_data)

In [15]:
for (i in 1:8) {
    coefi = coef(best_sub, id = i)
    pred_train <- train_pred_matrix[,names(coefi)] %*% coefi
    train_errors[i] = mean((Y_train - pred_train)^2)
    pred_test <- test_pred_matrix[,names(coefi)] %*% coefi
    test_errors[i] = mean((Y_test - pred_test)^2)
}

In [16]:
test_errors

[1] 0.2136666 0.2254541 0.2641099 0.2506773 0.2490325 0.2501823 0.2490105
[8] 0.2520347

First is the best

In [17]:
#10 fold 
pro_1_train = sample(1:nrow(prostate), nrow(prostate)*0.90)
pro_1_test = -pro_1_train
pro_1_train_data = prostate[pro_1_train, ]
pro_1_test_data = prostate[pro_1_test, ]

In [18]:
best_sub = regsubsets(train~.,data = pro_1_train_data,method = "exhaustive")
train_errors = rep(NA,8)
test_errors = rep(NA,8)
Y_train = pro_1_train_data$train
Y_test = pro_1_test_data$train
train_pred_matrix = model.matrix(train~., data = pro_1_train_data)

In [19]:
test_pred_matrix = model.matrix(train~., data = pro_1_test_data)

In [20]:
for (i in 1:8) {
    coefi = coef(best_sub, id = i)
    pred_train <- train_pred_matrix[,names(coefi)] %*% coefi
    train_errors[i] = mean((Y_train - pred_train)^2)
    pred_test <- test_pred_matrix[,names(coefi)] %*% coefi
    test_errors[i] = mean((Y_test - pred_test)^2)
}


In [21]:
test_errors

[1] 0.2339296 0.2526893 0.2230033 0.2371664 0.2430628 0.2701383 0.2668339
[8] 0.2708314

3rd is the best 

In [22]:
#bootstrap
#install.packages("bootstrap")

In [23]:
library(bootstrap)


Attaching package: ‘bootstrap’


The following object is masked from ‘package:pls’:

    crossval




In [24]:
best_sub = regsubsets(train~.,data = prostate,method="exhaustive")
reg_summary=summary(best_sub)
reg_summary

Subset selection object
Call: regsubsets.formula(train ~ ., data = prostate, method = "exhaustive")
9 Variables  (and intercept)
        Forced in Forced out
lcavol      FALSE      FALSE
lweight     FALSE      FALSE
age         FALSE      FALSE
lbph        FALSE      FALSE
svi         FALSE      FALSE
lcp         FALSE      FALSE
gleason     FALSE      FALSE
pgg45       FALSE      FALSE
lpsa        FALSE      FALSE
1 subsets of each size up to 8
Selection Algorithm: exhaustive
         lcavol lweight age lbph svi lcp gleason pgg45 lpsa
1  ( 1 ) " "    " "     "*" " "  " " " " " "     " "   " " 
2  ( 1 ) " "    " "     " " " "  " " " " "*"     "*"   " " 
3  ( 1 ) " "    " "     "*" " "  " " " " "*"     "*"   " " 
4  ( 1 ) " "    " "     "*" " "  " " "*" "*"     "*"   " " 
5  ( 1 ) " "    " "     "*" "*"  " " "*" "*"     "*"   " " 
6  ( 1 ) " "    "*"     "*" "*"  " " "*" "*"     "*"   " " 
7  ( 1 ) " "    "*"     "*" "*"  "*" "*" "*"     "*"   " " 
8  ( 1 ) "*"    "*"     "*" "*"  "*" "

In [25]:
beta.fit <- function(X,Y){
	lsfit(X,Y)	
}

In [26]:
beta.predict <- function(fit, X){
	cbind(1,X)%*%fit$coef
}

In [27]:
sq.error <- function(Y,Yhat){
	(Y-Yhat)^2
}


In [28]:
select = reg_summary$outmat
error_store <- c()

In [29]:
for (i in 1:8){
	# Pull out the model
	temp <- which(select[i,] == "*")
	
	res <- bootpred(prostate[,temp], prostate$train, nboot = 50, theta.fit = beta.fit, theta.predict = beta.predict, err.meas = sq.error) 
	error_store <- c(error_store, res[[3]])
	
}

In [30]:
error_store

[1] 0.2143871 0.2166471 0.2152602 0.2146147 0.2160477 0.2211364 0.2244976
[8] 0.2321876

First is the best